# Model modifications

## Ecoli modifications

In [1]:
import cobra
import re
import os.path
import copy
import cobra.flux_analysis.variability
import gurobipy
from cobra import Reaction
from cobra import Metabolite

In [2]:
def new_version_model_to_old(model):
    modeltmp=model.copy()
    for metabolite in modeltmp.metabolites:
        metabolite.id = re.sub('_c$',r'[c]',metabolite.id)
        metabolite.id = re.sub('_p$',r'[p]',metabolite.id)
        metabolite.id = re.sub('_e$',r'[e]',metabolite.id)
    modeltmp.repair()
    cobra.io.save_matlab_model(modeltmp,'tmp.mat')
    modeltmp=cobra.io.load_matlab_model('tmp.mat')
    for metabolite in modeltmp.metabolites:
        metabolite.id = re.sub('__',r'_',metabolite.id)
        metabolite.compartment = ''
    modeltmp.repair()
    # To solve possible problems in changing names
    cobra.io.save_matlab_model(modeltmp,'tmp.mat')
    modeltmp=cobra.io.load_matlab_model('tmp.mat')
    # Replace brackets with compartment location (e.g. "[c]") in metabolite ids by '_' (e.g. "_c")
    for rxn in modeltmp.reactions:
        rxn.id = re.sub('_p$',r'(p)',rxn.id)
        rxn.id = re.sub('_c$',r'(c)',rxn.id)
        rxn.id = re.sub('_e$',r'(e)',rxn.id)
    # To solve possible problems in changing names
    modeltmp.repair()
    cobra.io.save_matlab_model(modeltmp,'tmp.mat')
    modeltmp=cobra.io.load_matlab_model('tmp.mat')
    os.remove("tmp.mat") 
    return(modeltmp)


In [3]:
def mat_to_comets(matInputFile,modelIn):
    model=modelIn
    # Open output file:
    with open(matInputFile+'.cmt', mode='w') as f:
        # Print the S matrix
        f.write("SMATRIX  "+str(len(model.metabolites))+"  "+str(len(model.reactions))+"\n")
        for x in range(len(model.metabolites)):
            for y in range(len(model.reactions)):
                if (model.metabolites[x] in model.reactions[y].metabolites):
                    coeff=model.reactions[y].get_coefficient(model.metabolites[x])
                    f.write("    "+str(x+1)+"   "+str(y+1)+"   "+str(coeff)+"\n")
        f.write("//\n")

        # Print the bounds
        f.write("BOUNDS  -1000  1000\n");
        for y in range(len(model.reactions)):
            lb=model.reactions[y].lower_bound
            up=model.reactions[y].upper_bound
            if lb< -1000:
                lb=-1000
            if up>1000:
                up=1000
            f.write("    "+str(y+1)+"   "+str(lb)+"   "+str(up)+"\n")
        f.write("//\n")

        # Print the objective reaction
        f.write('OBJECTIVE\n')
        for y in range(len(model.reactions)):
            if (model.reactions[y].id in str(model.objective.expression)):
                indexObj=y+1
        f.write("    "+str(indexObj)+"\n")
        f.write("//\n")
        # Print the biomass reaction
        f.write('BIOMASS\n')
        for y in range(len(model.reactions)):
            if (model.reactions[y].id in str(model.objective.expression)):
                indexObj=y+1
        f.write("    "+str(indexObj)+"\n")
        f.write("//\n")

        # Print metabolite names
        f.write("METABOLITE_NAMES\n")
        for x in range(len(model.metabolites)):
            f.write("    "+model.metabolites[x].id+"\n")
        f.write("//\n")

        # Print reaction names
        f.write("REACTION_NAMES\n")
        for y in range(len(model.reactions)):
            f.write("    "+model.reactions[y].id+"\n")
        f.write("//\n")

        # Print exchange reactions
        f.write("EXCHANGE_REACTIONS\n")
        for y in range(len(model.reactions)):
            if (model.reactions[y].id.find('EX_')==0):
                f.write(" "+str(y+1))
        f.write("\n//\n")


In [4]:
def run_comets_and_plots(working_dir,comets_dir):
    old_path=os.getcwd()
    os.chdir(working_dir)
    os.environ["GUROBI_COMETS_HOME"] = "/home/chanle/software/gurobi900/linux64/"
    with open("output1.txt", "w") as f:
        subprocess.call(['./comets_scr','comets_script_template'], stdout=f)
    subprocess.call('cp *log_template* ..',shell=True)
    os.chdir(comets_dir)
    subprocess.call(["../../Scripts/plot_biomassX3_vs_4mediaItem.sh 'template' 'sucr' 'nar' 'malon' 'T4hcinnm' '24.0' '3_strains' 'blue' 'cyan' 'black' 'darkmagenta' 'Ecoli' 'KT' 'Salbus'"],shell=True)
    from wand.image import Image as WImage
    img = WImage(filename='biomassX3_vs_sucr_nar_malon_T4hcinnm_template_plot.pdf')
    os.chdir(old_path)
    return(img)


In [5]:
def run_comets_and_plots_one_ecoli(working_dir,comets_dir):
    old_path=os.getcwd()
    os.chdir(working_dir)
    os.environ["GUROBI_COMETS_HOME"] = "/home/chanle/software/gurobi900/linux64/"
    with open("output1.txt", "w") as f:
        subprocess.call(['./comets_scr','comets_script_template'], stdout=f)
    subprocess.call('cp *log_template* ..',shell=True)
    os.chdir(comets_dir)
    subprocess.call(["../../Scripts/plot_biomass_vs_4mediaItem.sh 'template' 'sucr' 'cbl1' 'fru' 'T4hcinnm' '24.0' 'ecoli' 'blue' 'cyan' 'black' 'darkmagenta' 'Ecoli'"],shell=True)
    from wand.image import Image as WImage
    img = WImage(filename='biomass_vs_sucr_cbl1_fru_T4hcinnm_template_plot.pdf')
    os.chdir(old_path)
    return(img)

In [6]:
import requests
url='http://bigg.ucsd.edu/static/models/iJN1463.xml'
r=requests.get(url,allow_redirects=True)
open('iJN1463.xml','wb').write(r.content)

10702784

In [7]:
model3=cobra.io.read_sbml_model('iJN1463.xml')
model3.summary()

Using license file /home/chanle/software/gurobi900/linux64/gurobi.lic
Academic license - for non-commercial use only


Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.002477,0,0.00%
cl_e,EX_cl_e,0.002477,0,0.00%
cobalt2_e,EX_cobalt2_e,0.001782,0,0.00%
cu2_e,EX_cu2_e,0.001651,0,0.00%
fe2_e,EX_fe2_e,0.008608,0,0.00%
glc__D_e,EX_glc__D_e,6,6,99.99%
k_e,EX_k_e,0.0929,0,0.00%
mg2_e,EX_mg2_e,0.004129,0,0.00%
mn2_e,EX_mn2_e,0.001651,0,0.00%
mobd_e,EX_mobd_e,0.001913,0,0.00%


## Malonate production

In [8]:
malon_c= Metabolite('malon_c', compartment='c',formula='C3H2O4', name='Malonate')
malon_p= Metabolite('malon_p', compartment='p',formula='C3H2O4', name='Malonate')
malon_e= Metabolite('malon_e', compartment='e',formula='C3H2O4', name='Malonate')
    
reaction2=Reaction('MALONHY')
reaction2.name='malonyl-CoA thioesterase'
reaction2.lower_bound=0
reaction2.upper_bound=1000
reaction2.add_metabolites({model3.metabolites.get_by_id("h2o_c"):-1.0,
        model3.metabolites.get_by_id("malcoa_c"):-1.0,
        model3.metabolites.get_by_id("coa_c"):1.0,
        model3.metabolites.get_by_id("h_c"):1.0,
        malon_c:1.0})
model3.add_reaction(reaction2)
    
    
reaction3=Reaction('MALONpp')
reaction3.name='Malonate proton symport transport'
reaction3.lower_bound=0
reaction3.upper_bound=0
reaction3.add_metabolites({model3.metabolites.get_by_id("h_p"):-1.0,
        malon_p:-1.0,
        model3.metabolites.get_by_id("h_c"):1.0,
        model3.metabolites.get_by_id("malon_c"):1.0})
model3.add_reaction(reaction3)
    
    
reaction4=Reaction('MALONtex')
reaction4.name='Malonate transport via diffusion extracellular to periplasm'
reaction4.lower_bound=0
reaction4.upper_bound=0
reaction4.add_metabolites({model3.metabolites.get_by_id("malon_p"):-1.0,
        malon_e:1.0})
model3.add_reaction(reaction4)
   
reaction5=Reaction('EX_malon_e')
reaction5.name='malon exchange'
reaction5.lower_bound=0
reaction5.upper_bound=0
reaction5.add_metabolites({model3.metabolites.get_by_id("malon_e"):-1.0})
model3.add_reaction(reaction5)

## Fructose and o2 limits

In [9]:
model3.reactions.EX_glc__D_e.bounds=(0,0)
model3.reactions.EX_fru_e.bounds=(-8,0)
model3.reactions.FRUtex.bounds=(0,8)
model3.reactions.EX_o2_e.bounds=(-20,0)

## P-coumarate - medium

In [10]:
m=model3.medium
m["EX_T4hcinnm_e"]=10
m["EX_fru_e"]=10
model3.medium=m

## naringenin assembly

In [11]:
M4cmcoa_c=Metabolite('4cmcoa_c', compartment='c',formula='C30H38N7O18P3S', name='4_Coumaroyl_CoA')
narchal_c=Metabolite('narchal_c', compartment='c',formula='C15H12O5', name='Naringenin_chalcone')
nar_c=Metabolite('nar_c', compartment='c',formula='C15H12O5', name='Naringenin')
nar_p=Metabolite('nar_p', compartment='p',formula='C15H12O5', name='Naringenin')
nar_e=Metabolite('nar_e', compartment='e',formula='C15H12O5', name='Naringenin')

# Reactions to assemble naringenin

reaction1=Reaction("AS_C_4CMCOAS_FR")
reaction1.name="feruloyl coenzyme A synthetase AMP forming"
reaction1.lower_bound=0
reaction1.upper_bound=1000
reaction1.add_metabolites({model3.metabolites.get_by_id("T4hcinnm_c"):-1.0,
        model3.metabolites.get_by_id("atp_c"):-1.0,
        model3.metabolites.get_by_id("coa_c"):-1.0,
        M4cmcoa_c:1.0,
        model3.metabolites.get_by_id("amp_c"):1.0,
        model3.metabolites.get_by_id("ppi_c"):1.0})
model3.add_reaction(reaction1)

reaction2=Reaction("AS_C_CHALS1_FR")
reaction2.name="Chalcone synthase 1(Naringenin_chalcone)"
reaction2.lower_bound=0
reaction2.upper_bound=1000
reaction2.add_metabolites({model3.metabolites.get_by_id("4cmcoa_c"):-1.0,
        model3.metabolites.get_by_id("malcoa_c"):-3.0,
        model3.metabolites.get_by_id("h_c"):-3.0,
        narchal_c:1.0,
        model3.metabolites.get_by_id("co2_c"):3.0,
        model3.metabolites.get_by_id("coa_c"):4.0})
model3.add_reaction(reaction2)

reaction3=Reaction("AS_CHALIS1_FR")
reaction3.name="Chalcone isomerase 1 (Naringenin)"
reaction3.lower_bound=0
reaction3.upper_bound=1000
reaction3.add_metabolites({model3.metabolites.get_by_id("narchal_c"):-1.0,
                          nar_c:1.0})
model3.add_reaction(reaction3)



# Reaccion malon_c + ac_c + atp_c <=> ppi_c + malcoa_c +amp_c
reaction18=Reaction("matB")
reaction18.name="Acetyl-CoA:malonate CoA-synthetase"
reaction18.lower_bound=0
reaction18.upper_bound=1000
reaction18.add_metabolites({model3.metabolites.get_by_id("malon_c"):-1.0,
                           model3.metabolites.get_by_id("atp_c"):-1.0,
                           model3.metabolites.get_by_id("coa_c"):-1.0,
                            model3.metabolites.get_by_id("malcoa_c"):1.0,
                           model3.metabolites.get_by_id("ppi_c"):1.0,
                           model3.metabolites.get_by_id("amp_c"):1.0})

model3.add_reaction(reaction18)



# Naringenin transporters
reaction11=Reaction('naringenintex')
reaction11.name='naringenin transport via diffusion periplasm'
reaction11.lower_bound=-1000
reaction11.upper_bound=1000

reaction11.add_metabolites({nar_p:-1.0, nar_e:1.0})
model3.add_reaction(reaction11)

reaction12=Reaction('naringenintpp')
reaction12.name='naringenin transport via diffusion periplasm'
reaction12.lower_bound=-1000
reaction12.upper_bound=1000

reaction12.add_metabolites({model3.metabolites.get_by_id("nar_c"):-1.0, model3.metabolites.get_by_id("nar_p"):1.0})
model3.add_reaction(reaction12)


reaction13=Reaction('EX_nar_e')
reaction13.name='naringenin exchange'
reaction13.lower_bound=0
reaction13.upper_bound=1000
reaction13.add_metabolites({nar_e:-1.0})
model3.add_reaction(reaction13)



## Model to file

In [12]:
cobra.io.write_sbml_model(model3,"iJN1463_naringenin.xml")

## FVA to find the best Naringenin production

In [14]:
narPer=15
dictnarValue=cobra.flux_analysis.variability.flux_variability_analysis(model3,['EX_nar_e'],fraction_of_optimum=1-(narPer/100))
narLimit=dictnarValue['maximum']['EX_nar_e']

model3.reactions.get_by_id('naringenintex').bounds=(narLimit,1000)
model3.reactions.get_by_id('EX_nar_e').bounds=(narLimit,narLimit)
dictnarValue

,minimum,maximum
EX_nar_e,0.0,1.981662


In [15]:
model3.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
T4hcinnm_e,EX_T4hcinnm_e,10,9,65.22%
ca2_e,EX_ca2_e,0.004038,0,0.00%
cl_e,EX_cl_e,0.004038,0,0.00%
cobalt2_e,EX_cobalt2_e,0.002905,0,0.00%
cu2_e,EX_cu2_e,0.002692,0,0.00%
fe2_e,EX_fe2_e,0.01403,0,0.00%
fru_e,EX_fru_e,8,6,34.78%
k_e,EX_k_e,0.1515,0,0.00%
mg2_e,EX_mg2_e,0.006731,0,0.00%
mn2_e,EX_mn2_e,0.002692,0,0.00%


In [17]:
#cobra.io.write_sbml_model(model3,"iJN1463_naringenin.xml") FAIL
cobra.io.save_matlab_model(model3,"iJN1463_naringenin.mat")

In [ ]:
model_old=new_version_model_to_old(model3)
mat_to_comets('iJN1463_naringenin',model3)

Read LP format model from file /tmp/tmprcnkwy9b.lp
Reading time = 0.01 seconds
: 2161 rows, 5876 columns, 23346 nonzeros
